[View in Colaboratory](https://colab.research.google.com/github/upadhyayprakash/EIP/blob/master/PRAKASH_BATCH_3_ASSIGNMENT4B.ipynb)

**Assigment 4B of EIP Batch 3 - Prakash Upadhyay**</br>
**Task: ** Beat the Validation Score of **76.09%** accuracy.

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# Added by Prakash
#x_train = x_train.astype('float32')
#x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    #AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    #flat = Flatten()(AvgPooling)
    flat = Flatten()(relu)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

# START: Added Pooling & Dropout by Prakash
#BatchNorm = BatchNormalization()(input)
#relu = Activation('relu')(BatchNorm)
    
#Second_Conv2D = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)

#DropOut_Conv2D = Dropout(0.2)(Second_Conv2D)
#AvgPooling2D_Conv2D = AveragePooling2D(pool_size=(2,2))(DropOut_Conv2D)

# END: Added Pooling & Dropout by Prakash

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
#Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

#Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
#Last_Block = add_denseblock(Third_Block,  num_filter, dropout_rate)

output = output_layer(Third_Block)

In [22]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_41 (Conv2D)              (None, 32, 32, 12)   324         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_41 (BatchNo (None, 32, 32, 12)   48          conv2d_41[0][0]                  
__________________________________________________________________________________________________
activation_41 (Activation)      (None, 32, 32, 12)   0           batch_normalization_41[0][0]     
__________________________________________________________________________________________________
conv2d_42 

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [24]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 158s 3ms/step - loss: 1.6931 - acc: 0.3876 - val_loss: 1.7800 - val_acc: 0.4193
Epoch 2/50
19200/50000 [==========>...................] - ETA: 1:26 - loss: 1.3632 - acc: 0.5058

50000/50000 [==============================] - 149s 3ms/step - loss: 1.3115 - acc: 0.5285 - val_loss: 1.2740 - val_acc: 0.5548
Epoch 3/50
45568/50000 [==========================>...] - ETA: 12s - loss: 1.1324 - acc: 0.5950

50000/50000 [==============================] - 149s 3ms/step - loss: 1.1260 - acc: 0.5971 - val_loss: 1.2494 - val_acc: 0.5776
Epoch 4/50
50000/50000 [==============================] - 149s 3ms/step - loss: 1.0189 - acc: 0.6375 - val_loss: 1.2519 - val_acc: 0.6025
Epoch 5/50
 3328/50000 [>.............................] - ETA: 2:10 - loss: 0.9602 - acc: 0.6626

50000/50000 [==============================] - 149s 3ms/step - loss: 0.9351 - acc: 0.6665 - val_loss: 1.1478 - val_acc: 0.6341
Epoch 6/50
39552/50000 [======================>.......] - ETA: 29s - loss: 0.8704 - acc: 0.6920

50000/50000 [==============================] - 149s 3ms/step - loss: 0.8654 - acc: 0.6940 - val_loss: 1.0571 - val_acc: 0.6517
Epoch 7/50
50000/50000 [==============================] - 149s 3ms/step - loss: 0.8134 - acc: 0.7137 - val_loss: 0.8646 - val_acc: 0.7059
Epoch 8/50
 1920/50000 [>.............................] - ETA: 2:15 - loss: 0.7632 - acc: 0.7271

...

50000/50000 [==============================] - 151s 3ms/step - loss: 0.3588 - acc: 0.8708 - val_loss: 0.6432 - val_acc: 0.8066
Epoch 45/50
38656/50000 [======================>.......] - ETA: 32s - loss: 0.3472 - acc: 0.8750

50000/50000 [==============================] - 151s 3ms/step - loss: 0.3541 - acc: 0.8733 - val_loss: 0.6392 - val_acc: 0.8138
Epoch 46/50
50000/50000 [==============================] - 152s 3ms/step - loss: 0.3502 - acc: 0.8750 - val_loss: 0.6596 - val_acc: 0.8091
Epoch 47/50
 1536/50000 [..............................] - ETA: 2:17 - loss: 0.3236 - acc: 0.8880

50000/50000 [==============================] - 152s 3ms/step - loss: 0.3523 - acc: 0.8735 - val_loss: 0.7858 - val_acc: 0.7751
Epoch 48/50
38656/50000 [======================>.......] - ETA: 32s - loss: 0.3459 - acc: 0.8751

50000/50000 [==============================] - 152s 3ms/step - loss: 0.3484 - acc: 0.8743 - val_loss: 0.6492 - val_acc: 0.8122
Epoch 49/50
50000/50000 [==============================] - 153s 3ms/step - loss: 0.3446 - acc: 0.8751 - val_loss: 0.7668 - val_acc: 0.7884
Epoch 50/50
 1536/50000 [..............................] - ETA: 2:20 - loss: 0.2895 - acc: 0.8997

50000/50000 [==============================] - 153s 3ms/step - loss: 0.3410 - acc: 0.8777 - val_loss: 0.6275 - val_acc: 0.8159


In [25]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 13s 1ms/step
Test loss: 0.6274621968507766
Test accuracy: 0.8159


In [26]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model_1st_50.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model_1st_50.h5')